# Génération de la Frontière Efficiente de Markowitz

Ce notebook illustre l'utilisation de la méthode de scalarisation pour approximer la frontière efficiente d'un portefeuille d'actions du secteur Energie.


In [ ]:
import matplotlib.pyplot as plt
from scipy.constants import sigma
from scipy.optimize import minimize
import os

from level1.functions import *

# Charger les données
df = pd.read_csv('../datasets/Information_Technology.csv', index_col=0, parse_dates=True)

for file in os.listdir('../datasets/'):
    if file.endswith('.csv') and 'Information_Technology' not in file:
        temp_df = pd.read_csv(os.path.join('../datasets/', file), index_col=0, parse_dates=True)
        df = df.join(temp_df, how='inner')

In [ ]:
df

In [ ]:
# Calcul des rendements logarithmiques
returns = f_returns(df)

# Calcul des paramètres pour l'optimisation
mu = f_mu(returns)  # Annualisation (252 jours boursiers)
Sigma = f_sigma(returns)  # Annualisation de la matrice de covariance
num_assets = len(mu)

# Calcul des Paramètres d'Optimisation

Les rendements logarithmiques sont calculés comme :

$  r_t = \ln\left(\frac{P_t}{P_{t-1}}\right) $

Le vecteur des rendements moyens annualisés :

$ \mu = \frac{1}{T} \sum_{t=1}^T r_t \times 252 $

La matrice de covariance annualisée :

$ \Sigma = \frac{1}{T} \sum_{t=1}^T (r_t - \bar{r})(r_t - \bar{r})^T \times 252 $


# Méthode de résolution par scalarisation pour générer la frontière efficiente

Fonction rendement : $ F_1(w) = - (w^T \mu) $

Fonction risque : $ F_2(w) = w^T \Sigma w $

Fonction objectif scalarisée : $ F(w) = \lambda \cdot (w^T \Sigma w) - (1 - \lambda) \cdot (w^T \mu) $

In [ ]:
lambdas = np.linspace(0, 1, 50)  # 100 points entre 0 et 1
frontier_returns, frontier_volatilities, frontier_weights = optimize_portfolio(lambdas, mu, Sigma)

# Génération de la Frontière Efficiente

En faisant varier \(\lambda\) de 0 à 1, nous obtenons différents portefeuilles optimaux.


In [ ]:
# Tracer la frontière efficiente
plt.figure(figsize=(10, 6))
plt.scatter(frontier_volatilities, frontier_returns, c=lambdas, cmap='viridis', label='Frontière Efficiente')
plt.scatter(np.sqrt(np.diag(Sigma)), mu, c='red', marker='x', label='Actifs Individuels')
plt.colorbar(label='Lambda (Aversion au Risque)')
plt.xlabel('Risque (Volatilité Annuelle)')
plt.ylabel('Rendement Annuel Attendu')
plt.title('Frontière Efficiente de Markowitz - Niveau 1')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
print("Le portefeuille avec le rendement le plus élevé :")
max_return_index = np.argmax(frontier_returns)
print(f"Rendement : {frontier_returns[max_return_index]:.4f}, Volatilité : {frontier_volatilities[max_return_index]:.4f}")
weights = frontier_weights[max_return_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

print("\nLe portefeuille avec le risque le plus faible :")
min_risk_index = np.argmin(frontier_volatilities)
print(f"Rendement : {frontier_returns[min_risk_index]:.4f}, Volatilité : {frontier_volatilities[min_risk_index]:.4f}")
weights = frontier_weights[min_risk_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

# Autre approche

In [ ]:
def portfoliorisk(w, cov_matrix):
    """Calcule la variance du portefeuille (le risque)"""
    return w.T @ cov_matrix @ w

In [ ]:
# Contrainte : la somme des poids doit être égale à 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bornes : chaque poids est entre 0 et 1 (pas de vente à décourt)
bounds = tuple((0, 1) for _ in range(len(mu)))

# Contrainte de rendement minimum (elle sera ajoutée plus tard pour chaque valeur de r_min)
# Elle sera de la forme : {'type': 'ineq', 'fun': lambda w: w.T @ mu - r_min}

In [ ]:
# 1. Choisis une plage de rendements cibles (r_min)
#    Trouve le rendement minimum (portefeuille le plus sûr) et maximum (portefeuille le plus risqué) possibles.
#    Cela te donne les extrémités de ton front de Pareto.

# Exemple : crée 50 points de rendement cible entre ces deux extremes
target_returns = np.linspace(mu.min(), mu.max(), 50)

# 2. Lists pour stocker les résultats
optimal_risks = []
optimal_weights = []

# 3. Pour chaque rendement cible, résous le problème d'optimisation
for r_min in target_returns:
    # Ajouter la contrainte de rendement pour cette itération
    constraints_with_return = (
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'ineq', 'fun': lambda w: w.T @ mu - r_min}
    )

    # Faire une supposition initiale (répartition équitable)
    initial_guess = np.array([1/len(mu)] * len(mu))

    # Résoudre le problème de minimisation
    result = minimize(portfoliorisk, initial_guess,
                      args=(Sigma,), method='SLSQP',
                      bounds=bounds, constraints=constraints_with_return)

    # Si la résolution a réussi, stocke le risque optimal et les poids
    if result.success:
        optimal_risks.append(result.fun)
        optimal_weights.append(result.x)
    else:
        # Gérer les erreurs (cela peut arriver pour des rendements cibles trop extrêmes)
        optimal_risks.append(np.nan)
        optimal_weights.append(np.nan)

# Convertir les listes en arrays numpy
optimal_risks = np.array(optimal_risks)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(optimal_risks, target_returns, 'b-', linewidth=2.0, label='Frontière Efficiente')
plt.xlabel('Risque (Variance)')
plt.ylabel('Rendement Attendu')
plt.title('Frontière Efficiente de Markovitz')
plt.grid(True)
plt.legend()
plt.show()